# 1. Import and install

**<span style="color: rgb(224, 0, 0)">Important notes</span>**: we should add another field if running with docker-compose.  (docker-compose.yaml)

```bash
services:

alice:

... (some other settings)

cap_add:

  - NET_ADMIN


bob:

... (some other settings)

cap_add:

  - NET_ADMIN
```

In [1]:
import secretflow as sf
import spu
import os
import numpy as np
import jax.numpy as jnp
import jax
import jax.lax
import spu.utils.simulation as spsim
import spu.spu_pb2 as spu_pb2
from functools import partial

network_conf = {
    "parties": {
        "alice": {
            "address": "alice:8000",
        },
        "bob": {
            "address": "bob:8000",
        },
    },
}

party = os.getenv("SELF_PARTY", "alice")
sf.shutdown()
sf.init(
    address="127.0.0.1:6379",
    cluster_config={**network_conf, "self_party": party},
    log_to_driver=True,
)

alice's Output:

2024-06-23 08:48:37,781	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.23.0.2:6379...
2024-06-23 08:48:37,874	INFO worker.py:1724 -- Connected to Ray cluster.
2024-06-23 08:48:37.890 INFO api.py:233 [alice] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8000', 'bob': 'bob:8000'}, 'CURRENT_PARTY_NAME': 'alice', 'TLS_CONFIG': {}}
2024-06-23 08:48:38.502 INFO barriers.py:284 [alice] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=2943) 2024-06-23 08:48:38.499 INFO grpc_proxy.py:359 [alice] -- [Anonymous_job] ReceiverProxy binding port 8000, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCod

bob's Output:

2024-06-23 08:48:37,808	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.23.0.3:6379...
2024-06-23 08:48:37,915	INFO worker.py:1724 -- Connected to Ray cluster.
2024-06-23 08:48:37.936 INFO api.py:233 [bob] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8000', 'bob': 'bob:8000'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {}}
2024-06-23 08:48:38.554 INFO barriers.py:284 [bob] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=3800) 2024-06-23 08:48:38.548 INFO grpc_proxy.py:359 [bob] -- [Anonymous_job] ReceiverProxy binding port 8000, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["U

In [2]:
!yum install -y iproute-tc

alice's Output:

/usr/local/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Last metadata expiration check: 0:21:00 ago on Sun Jun 23 08:27:39 2024.
Package iproute-tc-6.2.0-5.0.1.an8.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!


bob's Output:

/usr/local/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Last metadata expiration check: 0:20:59 ago on Sun Jun 23 08:27:40 2024.
Package iproute-tc-6.2.0-5.0.1.an8.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!


# Simulation

In [3]:
# we know that dk is wrong when |x| is very small
# Let us try it. (we only show part here.)
# define some test function and data used in simulation
def test_square_and_sum_when_x_small(x):
    return jnp.sum(jnp.square(x))

x = np.array([1e-5] * 10)

In [4]:
# First, we run SPU with simulator
# Indeed, simulation can be run within single node.
# a. run with CHEETAH
sim_che = spsim.Simulator.simple(2, spu_pb2.ProtocolKind.CHEETAH, spu_pb2.FieldType.FM64)
spsim.sim_jax(sim_che, test_square_and_sum_when_x_small)(x)

alice's Output:

2024-06-23 08:48:39.740 INFO xla_bridge.py:863 [alice] -- [Anonymous_job] Unable to initialize backend 'cuda': 
2024-06-23 08:48:39.741 INFO xla_bridge.py:863 [alice] -- [Anonymous_job] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2024-06-23 08:48:39.742 INFO xla_bridge.py:863 [alice] -- [Anonymous_job] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2024-06-23 08:48:39.743 INFO xla_bridge.py:863 [alice] -- [Anonymous_job] Unable to initialize backend 'interpreter': jaxlib.xla_extension.Client: no constructor defined!


[2024-06-23 08:48:39.891] [info] [cheetah_mul.cc:321] CheetahMul uses 4 modulus for 64 bit input over 64 bit ring
[2024-06-23 08:48:39.892] [info] [cheetah_mul.cc:321] CheetahMul uses 4 modulus for 64 bit input over 64 bit ring
[2024-06-23 08:48:39.899] [info] [thread_pool.cc:30] Create a fixed thread pool with size 19


array(-1.5258789e-05, dtype=float32)

bob's Output:

2024-06-23 08:48:39.787 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'cuda': 
2024-06-23 08:48:39.788 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2024-06-23 08:48:39.789 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2024-06-23 08:48:39.790 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'interpreter': jaxlib.xla_extension.Client: no constructor defined!


[2024-06-23 08:48:39.932] [info] [cheetah_mul.cc:321] CheetahMul uses 4 modulus for 64 bit input over 64 bit ring
[2024-06-23 08:48:39.932] [info] [cheetah_mul.cc:321] CheetahMul uses 4 modulus for 64 bit input over 64 bit ring
[2024-06-23 08:48:39.940] [info] [thread_pool.cc:30] Create a fixed thread pool with size 19


array(-1.5258789e-05, dtype=float32)

In [5]:
# b. run with ABY3
# this time, we alse print some profile stats.
config_aby = spu.RuntimeConfig(
    protocol=spu_pb2.ProtocolKind.ABY3,
    field=spu.FieldType.FM64,
    fxp_fraction_bits=18,
    enable_hal_profile=True,
    enable_pphlo_profile=True,
)
sim_aby = spsim.Simulator(3, config_aby)
print(spsim.sim_jax(sim_aby, test_square_and_sum_when_x_small)(x))

alice's Output:

[2024-06-23 08:48:39.962] [info] [api.cc:163] [Profiling] SPU execution test_square_and_sum_when_x_small completed, input processing took 5.22e-07s, execution took 0.000550024s, output processing took 1.205e-06s, total time 0.000551751s.
[2024-06-23 08:48:39.962] [info] [api.cc:209] HLO profiling: total time 7.35e-07
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.constant, executed 1 times, duration 2.41e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.add, executed 5 times, duration 2.03e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.free, executed 3 times, duration 1.26e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.multiply, executed 1 times, duration 5.9e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.reduce, executed 1 times, duration 5.4e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:48:39.962] [info] [api.cc:212] - pphlo.

bob's Output:

[2024-06-23 08:48:40.007] [info] [api.cc:163] [Profiling] SPU execution test_square_and_sum_when_x_small completed, input processing took 4.61e-07s, execution took 0.000435953s, output processing took 1.218e-06s, total time 0.000437632s.
0.0
[2024-06-23 08:48:40.007] [info] [api.cc:209] HLO profiling: total time 9.6e-07
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphlo.constant, executed 1 times, duration 4.44e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphlo.add, executed 5 times, duration 2.1e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphlo.free, executed 3 times, duration 1.33e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphlo.multiply, executed 1 times, duration 5.9e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphlo.reduce, executed 1 times, duration 5.8e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:48:40.007] [info] [api.cc:212] - pphl

# Emulation

In [6]:
# delete all wan settings beforehand
!tc qdisc del dev eth0 root

alice's Output:

Error: Cannot delete qdisc with handle of zero.


bob's Output:

Error: Cannot delete qdisc with handle of zero.


In [7]:
!ping -c 4 bob

alice's Output:

PING bob (172.23.0.3) 56(84) bytes of data.
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=1 ttl=64 time=0.027 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=2 ttl=64 time=0.053 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=3 ttl=64 time=0.031 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=4 ttl=64 time=0.059 ms

--- bob ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3111ms
rtt min/avg/max/mdev = 0.027/0.042/0.059/0.015 ms


bob's Output:

PING bob (172.23.0.3) 56(84) bytes of data.
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=1 ttl=64 time=0.091 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=2 ttl=64 time=0.022 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=3 ttl=64 time=0.016 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=4 ttl=64 time=0.017 ms

--- bob ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3149ms
rtt min/avg/max/mdev = 0.016/0.036/0.091/0.032 ms


In [8]:
!ping -c 4 alice

alice's Output:

PING alice (172.23.0.2) 56(84) bytes of data.
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=1 ttl=64 time=0.102 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=2 ttl=64 time=0.012 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=3 ttl=64 time=0.010 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=4 ttl=64 time=0.012 ms

--- alice ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3158ms
rtt min/avg/max/mdev = 0.010/0.034/0.102/0.039 ms


bob's Output:

PING alice (172.23.0.2) 56(84) bytes of data.
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=1 ttl=64 time=0.034 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=2 ttl=64 time=0.038 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=3 ttl=64 time=0.029 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=4 ttl=64 time=0.043 ms

--- alice ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3091ms
rtt min/avg/max/mdev = 0.029/0.036/0.043/0.005 ms


In [9]:
# Emulation should be run from source in SPU, so we use Secretflow here to do the efficiency experiments.
# You can use the similar trick for emulation directly in SPU.

def compute_dk_func(x, eps=1e-6):
    return jax.lax.rsqrt(jnp.sum(jnp.square(x)) + eps)

x = np.random.rand(1_000_000)

In [10]:
# SPU settings
cluster_def = {
    'nodes': [
        {'party': 'alice', 'id': 'local:0', 'address': 'alice' + ':12945'},
        {'party': 'bob', 'id': 'local:1', 'address': 'bob' + ':12945'},
    ],
    'runtime_config': {
        # SEMI2K support 2/3 PC, ABY3 only support 3PC, CHEETAH only support 2PC.
        # pls pay attention to size of nodes above. nodes size need match to PC setting.
        'protocol': spu.spu_pb2.SEMI2K,
        'field': spu.spu_pb2.FM64
    },
}

alice_device = sf.PYU("alice")
bob_device = sf.PYU("bob")
spu_device = sf.SPU(cluster_def)

In [11]:
# first, load data to PYU
alice_data = alice_device(lambda x: x)(x)

In [12]:
# SPU may need some init, so we run this twice...
ret = spu_device(compute_dk_func)(alice_data)
sf.reveal(ret);

Now, we strict the wan settings for 100Mbit and 10ms delay (20ms RTT).

In [13]:
!tc qdisc add dev eth0 root handle 1: tbf rate 100mbit burst 128kb limit 10000
!tc qdisc add dev eth0 parent 1:1 handle 10: netem delay 10msec limit 8000

bob's Output:

Error: Specified qdisc kind is unknown.
Error: Failed to find specified qdisc.


alice's Output:

Error: Specified qdisc kind is unknown.
Error: Failed to find specified qdisc.


In [14]:
!ping -c 4 bob

bob's Output:

PING bob (172.23.0.3) 56(84) bytes of data.
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=1 ttl=64 time=0.025 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=2 ttl=64 time=0.013 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=3 ttl=64 time=0.023 ms
64 bytes from 2224ad06a363 (172.23.0.3): icmp_seq=4 ttl=64 time=0.028 ms

--- bob ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3120ms
rtt min/avg/max/mdev = 0.013/0.022/0.028/0.006 ms


alice's Output:

PING bob (172.23.0.3) 56(84) bytes of data.
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=1 ttl=64 time=0.027 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=2 ttl=64 time=0.034 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=3 ttl=64 time=0.044 ms
64 bytes from secretnote-bob-1.secretnote_default (172.23.0.3): icmp_seq=4 ttl=64 time=0.038 ms

--- bob ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3132ms
rtt min/avg/max/mdev = 0.027/0.035/0.044/0.009 ms


In [15]:
!ping -c 4 alice

bob's Output:

PING alice (172.23.0.2) 56(84) bytes of data.
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=1 ttl=64 time=0.027 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=2 ttl=64 time=0.034 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=3 ttl=64 time=0.037 ms
64 bytes from secretnote-alice-1.secretnote_default (172.23.0.2): icmp_seq=4 ttl=64 time=0.030 ms

--- alice ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3161ms
rtt min/avg/max/mdev = 0.027/0.032/0.037/0.003 ms


alice's Output:

PING alice (172.23.0.2) 56(84) bytes of data.
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=1 ttl=64 time=0.009 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=2 ttl=64 time=0.017 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=3 ttl=64 time=0.016 ms
64 bytes from a5ceb21c2bad (172.23.0.2): icmp_seq=4 ttl=64 time=0.017 ms

--- alice ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3082ms
rtt min/avg/max/mdev = 0.009/0.014/0.017/0.005 ms


In [16]:
ret = spu_device(compute_dk_func)(alice_data)
sf.reveal(ret);

In [17]:
!tc qdisc del dev eth0 root

bob's Output:

Error: Cannot delete qdisc with handle of zero.


alice's Output:

Error: Cannot delete qdisc with handle of zero.


In [18]:
# Homework
x = np.array([0.0, 0.2, 6.4, 3.0, 1.6, 12.0])
bins = np.array([0.0, 1.0, 2.5, 4.0, 10.0])
jnp.digitize(x, bins)

bob's Output:

Array([1, 1, 4, 3, 2, 5], dtype=int32)

alice's Output:

Array([1, 1, 4, 3, 2, 5], dtype=int32)

In [19]:
config_aby = spu.RuntimeConfig(
    protocol=spu_pb2.ProtocolKind.ABY3,
    field=spu.FieldType.FM64,
    fxp_fraction_bits=18,
    enable_hal_profile=True,
    enable_pphlo_profile=True,
)
sim_aby = spsim.Simulator(3, config_aby)
print(spsim.sim_jax(sim_aby, jnp.digitize)(x, bins))

bob's Output:

[2024-06-23 08:48:57.118] [info] [api.cc:163] [Profiling] SPU execution digitize completed, input processing took 5.5e-07s, execution took 0.087821865s, output processing took 5.51e-07s, total time 0.087822966s.
[1 1 4 3 2 5]
[2024-06-23 08:48:57.118] [info] [api.cc:209] HLO profiling: total time 4.9362000000000006e-05
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.free, executed 439 times, duration 1.9659e-05s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.reshape, executed 121 times, duration 5.458e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.less, executed 69 times, duration 3.89e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.dynamic_slice, executed 72 times, duration 3.144e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.add, executed 60 times, duration 3.125e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [ap

alice's Output:

[2024-06-23 08:48:57.118] [info] [api.cc:163] [Profiling] SPU execution digitize completed, input processing took 7.22e-07s, execution took 0.087801274s, output processing took 7.45e-07s, total time 0.087802741s.
[1 1 4 3 2 5]
[2024-06-23 08:48:57.118] [info] [api.cc:209] HLO profiling: total time 5.0784000000000014e-05
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.free, executed 439 times, duration 2.0416e-05s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.reshape, executed 121 times, duration 5.408e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.less, executed 69 times, duration 4.161e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.dynamic_slice, executed 72 times, duration 3.362e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [api.cc:212] - pphlo.add, executed 60 times, duration 3.268e-06s, send bytes 0 recv bytes 0
[2024-06-23 08:48:57.118] [info] [

In [21]:
# MPC-friendly jnp.digitize example
# 根据是否有right变量重载函数结果无效，只能用不同的函数名
def my_digitize_no_endpoint(x, bins,right=False):
    # vectorize
    com = x.reshape(*x.shape, -1) >= bins
    # count the number of x that exceeds bins
    return jnp.sum(com, axis=-1)
def my_digitize_with_endpoint(x, bins,right=True):
    # vectorize
    com = x.reshape(*x.shape, -1) > bins
    # count the number of x that exceeds bins
    return jnp.sum(com, axis=-1)

res_without_right = spsim.sim_jax(sim_aby, my_digitize_no_endpoint)(x, bins,False)
res_with_right = spsim.sim_jax(sim_aby, my_digitize_with_endpoint)(x, bins,True)
print(f"res of digitize: {res_without_right}, jnp.digitize: {jnp.digitize(x, bins, False)}")
print(f"res of digitize: {res_with_right},  jnp.digitize:{jnp.digitize(x, bins, True)}")
print("-"*60)

alice's Output:

[2024-06-23 08:50:09.726] [info] [api.cc:163] [Profiling] SPU execution my_digitize_no_endpoint completed, input processing took 1.29e-06s, execution took 0.001513132s, output processing took 1.755e-06s, total time 0.001516177s.
[2024-06-23 08:50:09.726] [info] [api.cc:209] HLO profiling: total time 1.1920000000000002e-06
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.constant, executed 1 times, duration 3.27e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.free, executed 7 times, duration 3.12e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.add, executed 3 times, duration 1.75e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.convert, executed 2 times, duration 9.9e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.broadcast, executed 2 times, duration 9.4e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] -

bob's Output:

[2024-06-23 08:50:09.726] [info] [api.cc:163] [Profiling] SPU execution my_digitize_no_endpoint completed, input processing took 7.07e-07s, execution took 0.001461329s, output processing took 1.737e-06s, total time 0.001463773s.
[2024-06-23 08:50:09.726] [info] [api.cc:209] HLO profiling: total time 1.1380000000000002e-06
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.free, executed 7 times, duration 3.41e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.constant, executed 1 times, duration 2.27e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.add, executed 3 times, duration 2.14e-07s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.broadcast, executed 2 times, duration 9.6e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] - pphlo.convert, executed 2 times, duration 9.2e-08s, send bytes 0 recv bytes 0
[2024-06-23 08:50:09.726] [info] [api.cc:212] -